# Energy Estimator Tool
## Introduction
This is a port of the enVerid COVID-19 Energy Estimator 2 Excel Spreadsheet to a Jupyter Notebook to better understand the calculation methods the spreadsheet uses.

The spreadsheet must be present in the same folder folder as this file for the reference tables to be read.

### Dependencies
- [pandas](https://pypi.org/project/pandas/)
- [openpyxl](https://pypi.org/project/openpyxl/)
- [ipywidgets](https://pypi.org/project/ipywidgets/)
- [tabulate](https://pypi.org/project/tabulate/)

## Reference Tables
The same reference tables are used in this notebook.

The following code imports the reference tables. Run it once before any other cell.

In [1]:
import pandas as pd

SPREADSHEET_PATH = "enVerid COVID-19 Energy Estimator 2.xlsx"

filters = pd.read_excel(
    SPREADSHEET_PATH, "Table 2 - Filtration Info", skiprows=2, skipfooter=4, index_col=1
).dropna(axis="columns")

# ASHRAE 62.1 2016 Outdoor Air rates (ref. table 6-1)
# TODO: update to ASHRAE 62.1 2022
oa_rates = pd.read_excel(
    SPREADSHEET_PATH,
    "Table 3 - ASHRAE 62.1 OA Rates",
    header=1,
    index_col=0,
    names=["RPeople", "RArea"],
)

# TODO: replace this with custom data for any city
operation_info_126 = pd.read_excel(
    SPREADSHEET_PATH, "Table 1 - Operational Info", header=3, nrows=21, index_col=1
)

operation_info_247 = pd.read_excel(
    SPREADSHEET_PATH, "Table 1 - Operational Info", header=27, nrows=21, index_col=1
)

## Calculation Classes

The following classes receive user inputs and calculate appropriate outputs. These classes summarize the calculations performed by the enVerid COVID Energy Estimator spreadsheet.

In [2]:
from dataclasses import dataclass


@dataclass
class GeneralInputs:
    """Class for keeping track of general inputs of building parameters."""

    city: str
    space_type: str
    floor_area: float  # sq ft
    avg_ceil_ht: float  # ft
    occupancy: float
    sys_vent_efficiency: float
    oa_calc_method: str
    outside_airflow_override = None

    def __post_init__(self):
        # Approximate supply airflow with floor area. Overwrite if needed.
        self.total_supply_airflow = self.floor_area  # CFM
        self.volume = self.floor_area * self.avg_ceil_ht  # cu ft

    def vrp(self) -> float:
        """Calculate required airflow in CFM according to VRP."""
        occ_component = self.occupancy * oa_rates["RPeople"].loc[self.space_type]  # CFM
        area_component = self.floor_area * oa_rates["RArea"].loc[self.space_type]  # CFM
        return (occ_component + area_component) / self.sys_vent_efficiency

    def outside_airflow(self) -> float:
        """Calculate required outside airflow (CFM) given the OA calculation method."""
        match self.oa_calc_method:
            case "VRP":
                return self.vrp()
            case "VRP+30%":
                return self.vrp() * 1.3
            case "IAQP":
                return self.floor_area * 0.05  # 0.05 CFM required for each square foot?
            case "100% OA":
                return self.total_supply_airflow
            case "Other":
                return self.outside_airflow_override

    def outside_ach(self) -> float:
        """Calculate the outside air changes per hour."""
        air_changes_per_min = self.outside_airflow() / self.volume  # /min.
        return air_changes_per_min * 60  # /h

    def electricity_rate(self) -> float:
        return operation_info_126["Estimated Blended Electricity Rate ($/kWh)"].loc[
            self.city
        ]

In [3]:
@dataclass
class VentilationEnergyInputs:
    """Class for keeping track of ventilation energy inputs and costs."""

    oa_cooling_src = "Electricity"  # TODO: add chilled water cooling
    oa_heating_src: str
    hrs_per_day_operation: float
    days_per_wk_operation: float
    cop = 3.0
    heating_efficiency = 1.0

    def hrs_per_wk_operation(self) -> float:
        """Calculate the hours per week the building in operation."""
        return self.hrs_per_day_operation * self.days_per_wk_operation

    def heating_energy_rate(self, gen: GeneralInputs) -> tuple[float | str, str]:
        """Calculate the cost of heating and a string of the pricing unit."""
        match self.oa_heating_src:
            case "Electricity":
                return gen.electricity_rate(), "$/kWh (blended)"
            case "Gas":
                heating_rate = operation_info_126["Gas Rate ($/therm)"].loc[gen.city]
                return heating_rate, "$/therm"
            case "Steam":
                heating_rate = operation_info_126["Steam Rate ($/Mlb)"].loc[gen.city]
                return heating_rate, "$/mmBTU"

In [4]:
@dataclass
class EnergyRecoveryInputs:
    """Class to keep track of the energy recovery usage."""

    is_used: bool
    summer_effectiveness: float
    winter_effectiveness: float

    def __post_init__(self):
        """Fix recovery effectivness to 0 if energy recovery is not used."""
        if not self.is_used:
            self.summer_effectiveness = 0
            self.winter_effectiveness = 0

In [5]:
@dataclass
class IndoorAirFiltrationInputs:
    """Class to keep track of indoor air filtration inputs and costs."""

    is_used: bool
    merv: str
    fan_efficiency = 0.85
    mtr_efficiency = 0.85
    max_filt_airspeed = 500.0  # fpm
    filt_change_labor_cost = 17.00  # $  ($12 install + $5 disposal)

    def avg_pressure_drop(self) -> float:
        """Return the average pressure drop across the filter in inches water."""
        return filters["average pressure drop (in.w.c.)"].loc[self.merv]

    def est_filter_lifespan(self) -> float:
        """Return the average lifespan of the filter in months."""
        return filters["filter lifespan (months)"].loc[self.merv]

    def est_filter_cost(self) -> float:
        """Return the estimated cost ($) of a 24in. by 24in. filter."""
        return filters["cost"].loc[self.merv]

    def filtration_ach(self, bldg: GeneralInputs) -> float:
        """Return the air changes per hour supplied with recirculated air."""
        if self.is_used:
            recirc_airflow = bldg.total_supply_airflow - bldg.outside_airflow()  # CFM
        else:
            return 0
        filt_eff = filters["Filter Efficiency (%)"].loc[self.merv]
        return recirc_airflow / (bldg.volume) * filt_eff * 60  # /h

In [6]:
@dataclass
class AirCleanerInputs:
    """Class to keep track of air cleaner usage inputs and costs."""

    is_used: bool
    quantity: int  # Spreadsheet has 1 air cleaner per every 1000 sq ft
    supply_air_per_cleaner: float
    cadr_per_cleaner: float
    first_cost_per_cleaner: float  # Includes hardware + installation cost
    est_lifespan_before_maint = 12.  # months
    labor_cost_maintenance = 40.00  # $/unit
    matrl_cost_maintenance = 150.00  # $/unit

    def __post_init__(self):
        """Set air cleaning power and zero values if air cleaning not used."""
        self.power = self.supply_air_per_cleaner * 0.0003  # kW / unit

        if not self.is_used:
            self.quantity = 0
            self.supply_air_per_cleaner = 0
            self.cadr_per_cleaner = 0

    def additional_ach(self, bldg: GeneralInputs) -> float:
        """Calculate additional air changes per hour provided by air cleaners."""
        return self.cadr_per_cleaner * self.quantity / bldg.volume * 60

In [7]:
class OutsideAirVentilationCost:
    """Class to calculate annual cost and energy use from conditioning OA."""

    def __init__(
        self,
        bldg: GeneralInputs,
        ops: VentilationEnergyInputs,
        erv: EnergyRecoveryInputs,
    ):
        self.bldg = bldg
        self.ops = ops
        self.erv = erv

        if self.ops.hrs_per_wk_operation() < (24 * 7):
            # 12 hrs per day, 6 days per week weather data used
            self.ops_info = operation_info_126
            self.weekly_op_hours_baseline = 72
        else:
            # Use special weather data for 24/7 operation
            self.ops_info = operation_info_247
            self.weekly_op_hours_baseline = 168

    def cooling_oa_energy(self) -> float:
        """Calculate energy to cool outside air in kWh/yr."""
        # NOTE: The units for cooling energy data from operations table are truly kWh/cfm/yr.
        return (
            self.bldg.outside_airflow()
            * self.ops_info["Cooling energy (kWh/cfm)"].loc[self.bldg.city]
            * (1 - self.erv.summer_effectiveness)
            / (self.ops.cop / 3)  # TODO: Why is COP divided by 3 for energy usage?
            * (self.ops.hrs_per_wk_operation() / self.weekly_op_hours_baseline)
        )

    def annual_cooling_oa_cost(self) -> float:
        """Calculate cost to cool outside air in $/yr."""
        return self.cooling_oa_energy() * self.ops.electricity_rate(self.bldg.city)

    def heating_oa_energy(self) -> tuple[float, str]:
        """Calculate energy to heat outside air in appropriate energy units per year."""
        match self.ops.oa_heating_src:
            case "Electricity":
                heating_energy = self.ops_info["Heating energy (kWh/cfm)"]
                unit = "kWh/yr"
            case "Gas":
                heating_energy = self.ops_info["Heating energy (therms/cfm)"]
                unit = "therm/yr"
            case "Steam":
                heating_energy = self.ops_info["Heating energy (mmBTU/cfm)"]
                unit = "mmBTU/yr"

        return (
            self.bldg.outside_airflow()
            * heating_energy.loc[self.bldg.city]
            * (1 - self.erv.winter_effectiveness)
            / self.ops.heating_efficiency
            * (self.ops.hrs_per_wk_operation() / self.weekly_op_hours_baseline),
            unit,
        )

    def annual_heating_oa_cost(self) -> tuple[float, str]:
        """Calculate cost to heat outside air in $/yr."""
        energy_rate, _ = self.ops.heating_energy_rate()
        return self.heating_oa_energy() * energy_rate

In [8]:
from math import ceil


class IndoorAirFiltrationCost:
    """Class to calculate annual cost and energy use from filtering indoor air."""

    def __init__(
        self,
        bldg: GeneralInputs,
        ops: VentilationEnergyInputs,
        filt: IndoorAirFiltrationInputs,
    ):
        self.bldg = bldg
        if filt.is_used:
            self.recirc_airflow = bldg.total_supply_airflow - bldg.outside_airflow()
        else:
            self.recirc_airflow = 0
        self.ops = ops
        self.filt = filt

    def filtration_fan_energy_load(self) -> float:
        """Calculate power load on the indoor air filtration fan in kW."""
        if not self.filt.is_used:
            return 0
        recirc_airflow_m3_per_h = self.recirc_airflow * 1.699
        recirc_airflow_m3_per_s = recirc_airflow_m3_per_h / 3600
        avg_press_drop_pa = self.filt.avg_pressure_drop() * 248.84
        fan_load = (
            recirc_airflow_m3_per_s
            * avg_press_drop_pa
            / (self.filt.fan_efficiency * self.filt.mtr_efficiency)  # W
        )
        return fan_load / 1000  # kW

    def annual_fan_energy(self) -> float:
        """Calculate energy use by filtration fan in kWh/yr."""
        return self.filtration_fan_energy_load() * self.ops.hrs_per_wk_operation() * 52

    def annual_fan_energy_cost(self) -> float:
        """Calculate cost due to energy used by the filtration fan in $/yr."""
        return self.annual_fan_energy() * self.ops.electricity_rate(self.bldg.city)

    def required_filter_quantity(self, filter_area_ft2) -> int:
        """Calculate number of filters needed with a given filter area in sq ft."""
        max_filt_airspeed_m_per_s = self.filt.max_filt_airspeed / 196.85
        recirc_airflow_m3_per_s = self.recirc_airflow * 1.699 / 3600
        req_filt_area_m2 = recirc_airflow_m3_per_s / max_filt_airspeed_m_per_s
        req_filt_area = req_filt_area_m2 * 10.7639  # sq ft

        # NOTE: The spreadsheet neglects this calculation, but the paper it references
        # (doi: 10.1016/j.buildenv.2013.08.025) says this is the procedure. As a result,
        # the spreadsheet costs are 4x higher than they should be for filter replacement costs!
        return ceil(req_filt_area / filter_area_ft2)

    def annual_material_cost(self) -> float:
        """Calculate cost due to replacing filters in $/yr."""
        changes_per_year = 12 / self.filt.est_filter_lifespan()
        # NOTE: Filter cost table uses 24" x 24" filters (4 sq ft)
        req_filters = self.required_filter_quantity(4)
        return changes_per_year * self.filt.est_filter_cost() * req_filters

    def annual_labor_cost(self) -> float:
        """Calculate cost due to labor involved in filter replacements in $/yr."""
        changes_per_year = 12 / self.filt.est_filter_lifespan()
        req_filters = self.required_filter_quantity(4)

        return changes_per_year * self.filt.filt_change_labor_cost * req_filters

    def annual_total_cost(self) -> float:
        """Calculate total cost of operating indoor air filtration in $/yr."""
        return (
            self.annual_fan_energy_cost()
            + self.annual_material_cost()
            + self.annual_labor_cost()
        )

In [9]:
class AirCleanerCost:
    """Class to calculate annual cost and energy consumption from air cleaners."""

    def __init__(
        self, bldg: GeneralInputs, ops: VentilationEnergyInputs, clean: AirCleanerInputs
    ):
        self.bldg = bldg
        self.ops = ops
        self.clean = clean

    def annual_energy(self) -> float:
        """Calculate the energy consumption of air cleaners in kWh/yr."""
        return (
            self.clean.power
            * self.clean.quantity
            * (self.ops.hrs_per_wk_operation() * 52)
        )

    def annual_energy_cost(self) -> float:
        """Calculate cost of air cleaner energy use in $/yr."""
        return self.annual_energy() * self.ops.electricity_rate(self.bldg.city)

    def annual_material_cost(self) -> float:
        """Calculate cost of air cleaner maintenance materials in $/yr."""
        return (
            (12 / self.clean.est_lifespan_before_maint)
            * self.clean.matrl_cost_maintenance
            * self.clean.quantity
        )

    def annual_labor_cost(self) -> float:
        """Calcaulate cost of air clean maintenance labor in $/yr."""
        return (
            (12 / self.clean.est_lifespan_before_maint)
            * self.clean.labor_cost_maintenance
            * self.clean.quantity
        )

    def annual_total_cost(self) -> float:
        """Calculate total cost of operating air cleaners in $/yr."""
        return (
            self.annual_energy_cost()
            + self.annual_material_cost()
            + self.annual_labor_cost()
        )

In [10]:
class EffectiveACH:
    """Class for calculating the combined effects of air changes."""

    def __init__(
        self,
        bldg: GeneralInputs,
        filt: IndoorAirFiltrationInputs,
        clean: AirCleanerInputs,
    ):
        self.oa_ach = bldg.outside_ach()
        self.filt_ach = filt.filtration_ach()
        self.additional_ach = clean.additional_ach()

In [11]:
class AnnualCostSummary:
    """Class for summarizing annual costs."""

    def __init__(
        self,
        ops_cost: OutsideAirVentilationCost,
        filt_cost: IndoorAirFiltrationCost,
        clean_cost: AirCleanerCost,
    ):
        self.oa_vent_cost = (
            ops_cost.annual_cooling_oa_cost() + ops_cost.annual_heating_oa_cost()
        )
        self.filter_energy_maint_cost = filt_cost.annual_total_cost()
        self.cleaner_energy_maint_cost = clean_cost.annual_total_cost()

        self.total_annual_cost = (
            self.oa_vent_cost
            + self.filter_energy_maint_cost
            + self.cleaner_energy_maint_cost
        )

In [12]:
class AnnualCarbonEmissions:
    """Class to calculate carbon emissions from building operations."""

    # NOTE: The enVerid spreadsheet makes a mistake here!
    # The electricity emission factor they use is 7.09E-4 t CO2 / kWh.
    # That factor is for electricity reductions!
    # We are concerned with emissions associated with consumption, not reduction.
    # See https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references

    ELECTRICITY_EMISSION_FACTOR = 4.33e-4  # t CO2e / kWh
    NG_EMISSION_FACTOR = 0.0053  # t CO2e / therm
    STEAM_EMISSION_FACTOR = 0.053  # t CO2e / mmBTU  (assumes perfect efficiency)

    def __init__(
        self,
        ops: VentilationEnergyInputs,
        ops_cost: OutsideAirVentilationCost,
        filt_cost: IndoorAirFiltrationCost,
        clean_cost: AirCleanerCost,
    ):
        self.ops = ops
        self.ops_cost = ops_cost
        self.annual_filt_energy = filt_cost.annual_fan_energy()
        self.annual_cleaner_energy = clean_cost.annual_energy()

    def cooling_co2_tons(self) -> float:
        """Calculate metric tons of CO2e produced annually from cooling."""
        return self.ops_cost.cooling_oa_energy() * self.ELECTRICITY_EMISSION_FACTOR

    def heating_co2_tons(self) -> float:
        """Calculate metric tons of CO2e produced annually from heating."""
        match self.ops.oa_heating_src:
            case "Electricity":
                return (
                    self.ops_cost.heating_oa_energy()[0]
                    * self.ELECTRICITY_EMISSION_FACTOR
                )
            case "Gas":
                return self.ops_cost.heating_oa_energy()[0] * self.NG_EMISSION_FACTOR
            case "Steam":
                return self.ops_cost.heating_oa_energy()[0] * self.STEAM_EMISSION_FACTOR

    def oa_ventilation_tons(self) -> float:
        """Calculate metric tons of CO2e produced annually from heating and cooling."""
        return self.cooling_co2_tons() + self.heating_co2_tons()

    def filter_fan_energy_tons(self) -> float:
        """Calculate metric tons of CO2e produced annually from filtration fan."""
        return self.annual_filt_energy * self.ELECTRICITY_EMISSION_FACTOR

    def air_cleaner_energy_tons(self) -> float:
        """Calculate metric tons of CO2e produced annually from air cleaners."""
        return self.annual_cleaner_energy * self.ELECTRICITY_EMISSION_FACTOR

    def total_emission_tons(self) -> float:
        """Calculate metric tons of CO2e produced annually from all sources."""
        return (
            self.oa_ventilation_tons()
            + self.filter_fan_energy_tons()
            + self.air_cleaner_energy_tons()
        )

## Inputs Section

In [13]:
from ipywidgets import *
from IPython.display import display


def apply_scaling(*args):
    """Applies desired style to widgets passed in."""
    for w in args:
        w.style = {"description_width": "initial"}
        w.layout = Layout(width="auto")


city_list = Dropdown(
    options=operation_info_126.index.values,
    value="Raleigh, NC",
    description="Representative City:",
)
space_list = Dropdown(
    options=oa_rates.index.values, value="Office space", description="Space type:"
)
floor_area_input = BoundedFloatText(
    value=50000, description="Floor area (sq ft):", max=1e7, min=1000
)
avg_height_input = BoundedFloatText(
    value=10, description="Average ceiling height (ft):", max=100, min=8
)
occupant_input = BoundedIntText(value=250, description="Occupancy:", min=10, max=1e4)
supply_air_input = HBox(
    [
        BoundedFloatText(
            value=50000,
            max=1e7,
            min=150,
            description="Total supply airflow (CFM):",
            style={"description_width": "initial"},
        ),
        Label(" If you don't know, assume 1 CFM per sq ft."),
    ]
)
efficiency_input = FloatSlider(
    value=0.75, description="System ventilation efficiency:", min=0, max=1.0, step=0.01
)
method_input = Dropdown(
    options=["VRP", "VRP+30%", "IAQP", "100% OA", "Other"],
    description="OA Calculation Method:",
)

widget_list = [
    city_list,
    space_list,
    floor_area_input,
    avg_height_input,
    occupant_input,
    supply_air_input,
    efficiency_input,
    method_input,
]

apply_scaling(*widget_list)
display(*widget_list)

Dropdown(description='Representative City:', index=13, layout=Layout(width='auto'), options=('Atlanta, GA', 'B…

Dropdown(description='Space type:', index=52, layout=Layout(width='auto'), options=('Art classroom', 'Auditori…

BoundedFloatText(value=50000.0, description='Floor area (sq ft):', layout=Layout(width='auto'), max=10000000.0…

BoundedFloatText(value=10.0, description='Average ceiling height (ft):', layout=Layout(width='auto'), min=8.0,…

BoundedIntText(value=250, description='Occupancy:', layout=Layout(width='auto'), max=10000, min=10, style=Desc…

FloatSlider(value=0.75, description='System ventilation efficiency:', layout=Layout(width='auto'), max=1.0, st…

Dropdown(description='OA Calculation Method:', layout=Layout(width='auto'), options=('VRP', 'VRP+30%', 'IAQP',…

In [14]:
general_input = GeneralInputs(
    city_list.value,
    space_list.value,
    floor_area_input.value,
    avg_height_input.value,
    occupant_input.value,
    efficiency_input.value,
    method_input.value,
)

outside_air_input = FloatText(
    value=general_input.outside_airflow(),
    description="Outside Airflow (CFM):",
    disabled=method_input.value != "Other",
)


@interact(x=outside_air_input, g=fixed(general_input))
def set_outside_air(x, g: GeneralInputs):
    g.outside_airflow_override = x
    out_widget = FloatText(
        g.outside_ach(), description="Outside air ACH (/h):", disabled=True
    )
    apply_scaling(out_widget)
    return out_widget


apply_scaling(outside_air_input)

interactive(children=(FloatText(value=5666.666666666667, description='Outside Airflow (CFM):', disabled=True),…

In [15]:
ventilation_inputs = VentilationEnergyInputs(None, 12, 6)

oa_cool_src_dropdown = Dropdown(
    options=["Electricity"], description="OA Ventilation Cooling Source"
)
oa_heat_src_dropdown = Dropdown(
    options=["Electricity", "Gas", "Steam"], description="OA Ventilation Heating Source"
)
hrs_per_day_input = BoundedFloatText(
    12.0, description="Hours per day of building operation", max=24, min=1
)
days_per_wk_input = BoundedIntText(
    6, description="Days per week of building operation", max=7, min=1
)
cop_input = FloatSlider(3.0, description="COP", max=5, min=0)
heat_eff_input = FloatSlider(
    1.0, description="Heating Efficiency", max=1, min=0.01, step=0.01
)


@interact(src=oa_cool_src_dropdown, vent=fixed(ventilation_inputs))
def set_oa_cool_source(src: str, vent: VentilationEnergyInputs):
    vent.oa_cooling_src = src


@interact(src=oa_heat_src_dropdown, vent=fixed(ventilation_inputs))
def set_oa_heat_source(src: str, vent: VentilationEnergyInputs):
    vent.oa_heating_src = src


@interact(hrs=hrs_per_day_input, vent=fixed(ventilation_inputs))
def set_hrs_per_day(hrs: float, vent: VentilationEnergyInputs):
    vent.hrs_per_day_operation = hrs


@interact(days=days_per_wk_input, vent=fixed(ventilation_inputs))
def set_days_per_wk(days: int, vent: VentilationEnergyInputs):
    vent.days_per_wk_operation = float(days)


@interact(cop=cop_input, vent=fixed(ventilation_inputs))
def set_cop(cop: float, vent: VentilationEnergyInputs):
    vent.cop = cop


@interact(heat_eff=heat_eff_input, vent=fixed(ventilation_inputs))
def set_heat_eff(heat_eff: float, vent: VentilationEnergyInputs):
    vent.heating_efficiency = heat_eff


elec_rate_disp = FloatText(
    round(general_input.electricity_rate(), 2),
    description="Blended Electricity Rate ($/kWh):",
    disabled=True,
)

(heat_rate, heat_unit) = ventilation_inputs.heating_energy_rate(general_input)
heat_rate_disp = Text(
    f"{heat_rate:.2f}",
    description=f"Heating energy rate ({heat_unit})",
    disabled=True,
)


def on_heat_src_change(_):
    """Update the value of the heating energy cost field."""
    heat_rate, heat_unit = ventilation_inputs.heating_energy_rate(general_input)
    if type(heat_rate) is not str:
        heat_rate = f"{heat_rate:.2f}"

    heat_rate_disp.value = heat_rate
    heat_rate_disp.description = f"Heating energy rate ({heat_unit})"


oa_heat_src_dropdown.observe(on_heat_src_change, names="value")
display(elec_rate_disp, heat_rate_disp)

apply_scaling(
    oa_cool_src_dropdown,
    oa_heat_src_dropdown,
    hrs_per_day_input,
    days_per_wk_input,
    cop_input,
    heat_eff_input,
    elec_rate_disp,
    heat_rate_disp,
)

interactive(children=(Dropdown(description='OA Ventilation Cooling Source', options=('Electricity',), value='E…

interactive(children=(Dropdown(description='OA Ventilation Heating Source', options=('Electricity', 'Gas', 'St…

interactive(children=(BoundedFloatText(value=12.0, description='Hours per day of building operation', max=24.0…

interactive(children=(BoundedIntText(value=6, description='Days per week of building operation', max=7, min=1)…

interactive(children=(FloatSlider(value=3.0, description='COP', max=5.0), Output()), _dom_classes=('widget-int…

interactive(children=(FloatSlider(value=1.0, description='Heating Efficiency', max=1.0, min=0.01, step=0.01), …

FloatText(value=0.09, description='Blended Electricity Rate ($/kWh):', disabled=True)

Text(value='0.09', description='Heating energy rate ($/kWh (blended))', disabled=True)

## Energy Recovery Inputs

In [52]:
energy_recov_input = EnergyRecoveryInputs(False, 0.75, 0.6)

recov_use_input = Checkbox(
    value=False, description="Is energy recovery being used in this building?"
)

summer_eff_input = BoundedFloatText(
    None,
    description="Summer energy recovery effectiveness",
    min=0,
    max=1,
    step=0.1,
    layout=Layout(display="none"),
)

winter_eff_input = BoundedFloatText(
    None,
    description="Winter energy recovery effectiveness",
    min=0,
    max=1,
    step=0.1,
    layout=Layout(visibility="hidden"),
)


@interact(use=recov_use_input, erv=fixed(energy_recov_input))
def set_recov_use(use: bool, erv: EnergyRecoveryInputs):
    erv.is_used = use


merv_out = Output()


def on_recov_use_change(_):
    with merv_out:
        if recov_use_input.value:
            energy_recov_input.summer_effectiveness = summer_eff_input.value
            energy_recov_input.winter_effectiveness = winter_eff_input.value
            display(summer_eff_input, winter_eff_input)
        else:
            merv_out.clear_output()


recov_use_input.observe(on_recov_use_change, "value")

display(merv_out)
apply_scaling(recov_use_input, summer_eff_input, winter_eff_input)

interactive(children=(Checkbox(value=False, description='Is energy recovery being used in this building?'), Ou…

Output()

## Indoor Air Filtration Inputs

In [83]:
filt_input = IndoorAirFiltrationInputs(False, "HEPA")

recirc_use_input = Checkbox(
    value=False, description="Is filtration being utilized to clean recirculated air?"
)


@interact(use=recirc_use_input, filt=fixed(filt_input))
def set_recirc_use(use: bool, filt: IndoorAirFiltrationInputs):
    filt.is_used = use


merv_select_input = Dropdown(
    options=filters.index.values,
    value="HEPA",
    description="Indoor Air Filtration MERV",
)

fan_eff_input = BoundedFloatText(
    value=filt_input.fan_efficiency,
    min=0,
    max=1,
    step=0.1,
    description="Fan Efficiency:",
)
mtr_eff_input = BoundedFloatText(
    value=filt_input.mtr_efficiency,
    min=0,
    max=1,
    step=0.1,
    description="Motor Efficiency:",
)
max_airspeed_input = BoundedFloatText(
    value=filt_input.max_filt_airspeed,
    min=0,
    step=50,
    description="Maximum allowable filter airspeed (fpm)",
)
filt_labor_cost_input = BoundedFloatText(
    value=filt_input.filt_change_labor_cost,
    min=0,
    step=1,
    description="Labor cost of replacing a filter ($/filter)",
)

selection_display_out = Output()
stats_out = Output()


def update_stats_out():
    stats_out.clear_output()
    with stats_out:
        display(
            f"Average pressure drop across filter: {filt_input.avg_pressure_drop():.3f} in. w.g."
        )
        display(f"Estimated filter lifespan: {filt_input.est_filter_lifespan()} months")
        display(f"Filtration ACH {filt_input.filtration_ach(general_input)} /h")


def on_recirc_use_change(_):
    """Show or hide the filtration parameters depending on if filtration is used."""
    filt_input.is_used = recirc_use_input.value
    with selection_display_out:
        if recirc_use_input.value:
            display(
                merv_select_input,
                fan_eff_input,
                mtr_eff_input,
                max_airspeed_input,
                filt_labor_cost_input,
            )
            update_stats_out()
        else:
            selection_display_out.clear_output()
            stats_out.clear_output()


def on_merv_change(_):
    """Update the values of the filtration inputs when any value changes."""
    filt_input.merv = merv_select_input.value
    with stats_out:
        update_stats_out()


def on_param_change(_):
    """Update the values of filtration inputs for non-MERV changes."""
    filt_input.fan_efficiency = fan_eff_input.value
    filt_input.mtr_efficiency = mtr_eff_input.value
    filt_input.max_filt_airspeed = max_airspeed_input.value
    filt_input.filt_change_labor_cost = filt_labor_cost_input.value


# Have each filtration input watch for changes in value.
recirc_use_input.observe(on_recirc_use_change, "value")
merv_select_input.observe(on_merv_change, "value")
fan_eff_input.observe(on_param_change, "value")
mtr_eff_input.observe(on_param_change, "value")
max_airspeed_input.observe(on_param_change, "value")
filt_labor_cost_input.observe(on_param_change, "value")

display(selection_display_out, stats_out)
apply_scaling(
    recirc_use_input,
    merv_select_input,
    fan_eff_input,
    mtr_eff_input,
    max_airspeed_input,
    filt_labor_cost_input,
)

interactive(children=(Checkbox(value=False, description='Is filtration being used to clean recirculated air?')…

Output()

Output()

## Air Cleaner Inputs

In [88]:
clean_input = AirCleanerInputs(False, 50, 0, 0, 0)

clean_use_input = Checkbox(
    value=False,
    description="Are air cleaners being utilized to clean additional recirculated air?",
)


@interact(use=clean_use_input, clean=fixed(clean_input))
def set_clean_use(use: bool, clean: AirCleanerInputs):
    clean.is_used = use


quant_input = BoundedIntText(
    value=clean_input.quantity, min=1, step=1, description="Quantity of air cleaners"
)
supply_air_input = BoundedFloatText(
    value=clean_input.supply_air_per_cleaner,
    min=0,
    description="Air cleaner supply per air cleaner (CFM)",
)
cadr_input = BoundedFloatText(
    value=clean_input.cadr_per_cleaner,
    min=0,
    description="CADR per air cleaner (CFM)"
)
_ = jslink((supply_air_input, "value"), (cadr_input, "max"))  # CADR <= airflow value
first_cost_input = BoundedFloatText(
    value=clean_input.first_cost_per_cleaner,
    min=0,
    description="First cost per air cleaner"
)
power_input = BoundedFloatText(
    value=clean_input.power,
    min=0,
    description="Air cleaner power (kW/unit)"
)
lifespan_input = BoundedFloatText(
    value=clean_input.est_lifespan_before_maint,
    min=1,
    description="Estimated lifespan before maintenance (months)"
)
clean_labor_cost_input = BoundedFloatText(
    value=clean_input.labor_cost_maintenance,
    min=0,
    description="Labor cost of maintenance ($/unit)"
)
clean_matrl_cost_input = BoundedFloatText(
    value=clean_input.matrl_cost_maintenance,
    min=0,
    description="Material cost of maintenance ($/unit)"
)

selection_display_out = Output()
stats_out = Output()

apply_scaling(clean_use_input)

interactive(children=(Checkbox(value=False, description='Are air cleaners being used to clean additional recir…

### General Inputs

In [74]:
floor_area: float
ceil_height: float


def set_dimensions(x=50_000, y=10):
    global floor_area, ceil_height
    floor_area = x
    ceil_height = y


vol_slider = interact(set_dimensions, x=(1000, 100_000, 100), y=(7.5, 20, 0.25))
apply_layout(vol_slider, "Floor Area (sq ft)", 0)
apply_layout(vol_slider, "Avg ceiling height (ft)", 1)

interactive(children=(IntSlider(value=50000, description='x', max=100000, min=1000, step=100), FloatSlider(val…

NameError: name 'apply_layout' is not defined

In [ ]:
supply_airflow: float
vent_efficiency: float
oa_calc_method: str

CALC_METHODS = ["VRP", "VRP+30%", "IAQP", "100% OA", "Other"]


def set_supply_airflow(q=floor_area, n=0.75, calc="VRP"):
    global supply_airflow, vent_efficiency
    global oa_calc_method
    supply_airflow = q
    vent_efficiency = n
    oa_calc_method = calc


supply_airflow_slider = interact(
    set_supply_airflow, q=(150, 200_000), n=(0, 1.0, 0.01), calc=CALC_METHODS
)
apply_layout(supply_airflow_slider, "Total Supply Airflow", 0)
apply_layout(supply_airflow_slider, "Ventilation Efficiency", 1)
apply_layout(supply_airflow_slider, "OA Calculation Method", 2)

In [ ]:
gen_inputs = GeneralInputs(
    repr_city,
    space_type,
    floor_area,
    ceil_height,
    occupancy,
    vent_efficiency,
    oa_calc_method,
)


def set_outside_airflow(q=gen_inputs.outside_airflow()):
    global gen_inputs
    if gen_inputs.oa_calc_method == "Other":
        gen_inputs.outside_airflow_override = q
    return "Outside Air ACH", round(gen_inputs.outside_ach(), 2)


airflow_slider = interact(set_outside_airflow, q=(150, 200_000, 1))
apply_layout(airflow_slider, "Outside Airflow (CFM)")
if gen_inputs.oa_calc_method != "Other":
    airflow_slider.widget.children[0].disabled = True

### Results

Calculate required outside airflow

In [ ]:
from tabulate import tabulate


def vrp_calc():
    """Calculate required airflow according to the Ventilation Rate Procedure."""
    occupancy_component = occupancy * oa_rates["RPeople"].loc[space_type]  # CFM
    area_component = floor_area * oa_rates["RArea"].loc[space_type]  # CFM
    return (occupancy_component + area_component) / vent_efficiency  # CFM


if oa_calc_method == "VRP":
    outside_airflow = vrp_calc()  # CFM
elif oa_calc_method == "VRP+30%":
    outside_airflow = vrp_calc() * 1.3
elif oa_calc_method == "IAQP":
    outside_airflow = floor_area * 0.05  # TODO: find out why this 5% factor
elif oa_calc_method == "100% OA":
    outside_airflow = total_supply_airflow
else:
    outside_airflow = int(input("Input a value for outside airflow."))

print(f"Outside airflow: {outside_airflow:.2f} CFM")

Calculate outside air changes per hour

In [ ]:
avg_volume = avg_ceil_hgt * floor_area  # ft^3
outside_air_ach = outside_airflow / avg_volume * 60  # /h
print(f"Ouside air ACH: {outside_air_ach:.2f} /h")

## Outside air ventilation Energy & Operating Costs


In [ ]:
oa_cooling_src = None
oa_heating_src = None


def set_oa_cooling_src(x):
    global oa_cooling_src
    oa_cooling_src = x


def set_oa_heating_src(x):
    global oa_heating_src
    oa_heating_src = x


COOLING_SRCS = ["Electricity"]
oa_cool_dropdown = interact(set_oa_cooling_src, x=COOLING_SRCS)
oa_cool_dropdown.widget.children[0].description = "OA Ventilation Cooling Source"
oa_cool_dropdown.widget.children[0].style = {"description_width": "initial"}

HEATING_SRCS = ["Electricity", "Gas", "Steam"]
oa_heat_dropdown = interact(set_oa_heating_src, x=HEATING_SRCS)
oa_heat_dropdown.widget.children[0].description = "OA Ventilation Heating Source"
oa_heat_dropdown.widget.children[0].style = {"description_width": "initial"}

In [ ]:
hrs_per_day_bldg_operation = None
days_per_wk_bldg_operation = None


def set_hrs_per_day(x=12):
    global hrs_per_day_bldg_operation
    hrs_per_day_bldg_operation = x


def set_days_per_week(x=6):
    global days_per_wk_bldg_operation
    days_per_wk_bldg_operation = x


hrs_per_day_slider = interact(set_hrs_per_day, x=(8, 24))
hrs_per_day_slider.widget.children[
    0
].description = "Hours per day of building operation"
hrs_per_day_slider.widget.children[0].style = {"description_width": "initial"}

days_per_wk_slider = interact(set_days_per_week, x=(1, 7))
days_per_wk_slider.widget.children[
    0
].description = "Days per week of building operation"
days_per_wk_slider.widget.children[0].style = {"description_width": "initial"}

In [ ]:
COP = 3
HEATING_EFFICIENCY = 1.00

operating_hrs_per_wk = hrs_per_day_bldg_operation * days_per_wk_bldg_operation
elec_rate_disp = operation_info_126["Estimated Blended Electricity Rate ($/kWh)"].loc[
    repr_city
]

if oa_heating_src == "Electricity":
    heating_energy_rate = elec_rate_disp
    unit = "$/kWh (blended)"
elif oa_heating_src == "Gas":
    heating_energy_rate = operation_info_126["Gas Rate ($/therm)"].loc[repr_city]
    unit = "$/therm"
elif oa_heating_src == "Steam":
    heating_energy_rate = operation_info_126["Steam Rate ($/Mlb)"].loc[repr_city]
    unit = "$/mmBTU"

disp_table = [
    ["Electricity rate", elec_rate_disp, "$/kWh (blended)"],
    ["Heating energy rate", heating_energy_rate, unit],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Energy Recovery Inputs (optional)

In [ ]:
energy_recovery = False


if energy_recovery:
    # TODO: sliders that only appear if checkbox for energy recovery ticked
    summer_recovery_eff = float(input("Summer energy recovery effectiveness (0-1.0): "))
    winter_recovery_eff = float(input("Winter energy recovery effectivness (0-1.0): "))
else:
    summer_recovery_eff = 0
    winter_recovery_eff = 0

## Indoor Air Filtration Operating Cost & First Cost Inputs

In [ ]:
RECIRC_FILTRATION = True
MERV = "MERV 11"  # TODO: dropdown for filter types

avg_press_drop = filters["average pressure drop (in.w.c.)"].loc[MERV]  # in. wc
fan_eff = 0.85
mtr_eff = 0.85
max_filter_airspeed = 500  # fpm
est_filter_lifespan = filters["filter lifespan (months)"].loc[MERV]  # months
filter_replace_labor_cost = 5  # $/filter
filtration_ach = 5  # /hr

## Air Cleaner Operating Cost & First Cost Inputs

In [ ]:
AIR_CLEANERS = False


air_cleaner_quant = 50
air_cleaner_supply_air = 200  # CFM per air cleaner
air_cleaner_cadr = 200  # CFM per air cleaner

additional_ach = air_cleaner_cadr * air_cleaner_quant / avg_volume * 60

cleaner_first_cost = 2200  # $ per air cleaner. Includes hardware + install cost
air_cleaner_pwr = air_cleaner_supply_air * 0.0003  # kW per air cleaner
cleaner_est_lifetime_before_maint = 12  # months
cleaner_maint_labor_cost = 50  # $ per unit
cleaner_maint_material_cost = 150  # $ per unit

## Outside Air Ventilation Energy Consumption & Cost

In [ ]:
if operating_hrs_per_wk < (24 * 7):
    ops_info = operation_info_126
    op_hours_baseline = 72  # Operations info based on operating 12/6
else:
    ops_info = operation_info_247
    op_hours_baseline = 168  # Operations info based on operating 24/7

cooling_oa_energy = (
    outside_airflow
    * ops_info["Cooling energy (kWh/cfm)"].loc[repr_city]
    * (1 - summer_recovery_eff)
    / (COP / 3)
    * (operating_hrs_per_wk / op_hours_baseline)
)  # kWh / yr

oa_cooling_cost = cooling_oa_energy * elec_rate_disp  # $/yr

if oa_heating_src == "Electricity":
    heating_energy = ops_info["Heating energy (kWh/cfm)"]
    unit = "kWh/yr"
elif oa_heating_src == "Steam":
    heating_energy = ops_info["Heating energy (mmBTU/cfm)"]
    unit = "mmBTU/yr"
elif oa_heating_src == "Gas":
    heating_energy = ops_info["Heating energy (therms/cfm)"]
    unit = "therm/yr"

heating_oa_energy = (
    outside_airflow
    * heating_energy.loc[repr_city]
    * (1 - winter_recovery_eff)
    / HEATING_EFFICIENCY
    * (operating_hrs_per_wk / op_hours_baseline)
)  # (kWh | mmBTU | therm)/yr

oa_heating_cost = heating_oa_energy * heating_energy_rate  # $/yr

disp_table = [
    ["Cooling Outside Air Ventilation Energy", cooling_oa_energy, "kWh/yr"],
    ["Cooling Outside Air Energy Cost", oa_cooling_cost, "$/yr"],
    ["Heating Outside Air Ventilation Energy", heating_oa_energy, unit],
    ["Heating Outside Air Energy Cost", oa_heating_cost, "$/yr"],
]

tabulate(disp_table, floatfmt=",.2f", tablefmt="html")

## Indoor Air Filtration Energy Consumption and Cost

In [ ]:
from math import ceil

CUBIC_METER_PER_HOUR_PER_CFM = 1.699
PASCAL_PER_INCH_WATER = 248.84
METER_PER_SECOND_PER_FPM = 1 / 196.85
SQ_FT_PER_SQ_METER = 10.7639

# Determine cost of running the fan to filter air
if not RECIRC_FILTRATION:
    recirc_airflow = 0
    indoor_air_filt_fan_load = 0  # kW
else:
    recirc_airflow = total_supply_airflow - outside_airflow  # CFM
    recirc_airflow_m3_per_h = recirc_airflow * CUBIC_METER_PER_HOUR_PER_CFM  # m^3/h
    recirc_airflow_m3_per_s = recirc_airflow_m3_per_h / 60 / 60  # m^3/s

    avg_press_drop_pa = avg_press_drop * PASCAL_PER_INCH_WATER  # Pa
    fan_load = recirc_airflow_m3_per_s * avg_press_drop_pa / (fan_eff * mtr_eff)  # W
    indoor_air_filt_fan_load = fan_load / 1000  # kW

annual_filtration_energy = indoor_air_filt_fan_load * operating_hrs_per_wk * 52  # kW/yr
annual_filt_energy_cost = annual_filtration_energy * elec_rate_disp  # $/yr

# Determine the cost of replacing filters
baseline_filter_changes_per_year = 12 / filters["filter lifespan (months)"].loc[MERV]
max_filter_airspeed_m_per_s = max_filter_airspeed * METER_PER_SECOND_PER_FPM  # m/s
min_filter_area_m2 = recirc_airflow_m3_per_s / max_filter_airspeed_m_per_s  # m^2
min_filter_area = ceil(min_filter_area_m2 * SQ_FT_PER_SQ_METER)  # ft^2
material_cost_filt_replacement = (
    baseline_filter_changes_per_year * filters["cost"].loc[MERV] * min_filter_area
)  # $/yr
# TODO: understand this calculation. Is the filter cost the cost per square foot?
# TODO: understand https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7127325/. Cost per unit removal rate?
labor_cost_filt_replacement = (
    baseline_filter_changes_per_year * filter_replace_labor_cost * min_filter_area
)

annual_total_cost = (
    annual_filt_energy_cost
    + material_cost_filt_replacement
    + labor_cost_filt_replacement
)

disp_table = [
    ["Indoor air filtration fan energy load", indoor_air_filt_fan_load, "kW"],
    ["Annual fan filtration energy", annual_filtration_energy, "kW"],
    ["Material cost of filter replacement", material_cost_filt_replacement, "$/yr"],
    ["Labor cost of filter replacement", labor_cost_filt_replacement, "$/yr"],
    [
        "Annual filter fan energy and filter replacement costs",
        annual_total_cost,
        "$/yr",
    ],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Air Cleaner Operating Cost & First Cost Inputs

In [ ]:
if AIR_CLEANERS:
    air_cleaner_annual_energy = (
        air_cleaner_pwr * air_cleaner_quant * operating_hrs_per_wk * 52
    )  # kW / yr

    air_cleaner_annual_energy_cost = air_cleaner_annual_energy * elec_rate_disp
    air_cleaner_filter_matrl_cost = (
        cleaner_maint_material_cost
        * air_cleaner_quant
        * 12
        / cleaner_est_lifetime_before_maint
    )

    air_cleaner_maint_labor_cost = (
        cleaner_maint_labor_cost
        * air_cleaner_quant
        * 12
        / cleaner_est_lifetime_before_maint
    )

    annual_air_cleaner_total_cost = (
        air_cleaner_annual_energy_cost
        + air_cleaner_filter_matrl_cost
        + air_cleaner_filter_matrl_cost
    )

    air_cleaner_total_first_cost = cleaner_first_cost * air_cleaner_quant

    disp_table = [
        ["Air cleaner annual energy consumption", air_cleaner_annual_energy, "kW/yr"],
        ["Air cleaner annual energy cost", air_cleaner_annual_energy_cost, "$/yr"],
        ["Air cleaner filter material cost", air_cleaner_filter_matrl_cost, "$/yr"],
        ["Air cleaner maintenance labor cost", air_cleaner_maint_labor_cost, "$/yr"],
        ["Air cleaner total annual cost", annual_air_cleaner_total_cost, "$/yr"],
        ["Air cleaner first cost", air_cleaner_total_first_cost, "$/yr"],
    ]
else:
    air_cleaner_annual_energy = 0
    annual_air_cleaner_total_cost = 0

## Effective Air Changes per Hour

In [ ]:
effective_ach = outside_air_ach + filtration_ach + additional_ach  # /h

disp_table = [
    ["Outside air ACH", outside_air_ach, "/h"],
    ["Filtration ACH", filtration_ach, "/h"],
    ["Additional ACH", additional_ach, "/h"],
    ["Effective ACH", effective_ach, "/h"],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Annual Cost Summary

In [ ]:
annual_oa_vent_energy_cost = oa_cooling_cost + oa_heating_cost
total_annual_cost = (
    annual_filt_energy_cost + annual_total_cost + annual_air_cleaner_total_cost
)

disp_table = [
    ["Annual outside air ventilation energy cost", annual_oa_vent_energy_cost, "$/yr"],
    [
        "Annual indoor air filter fan energy and filter replacement costs",
        annual_total_cost,
        "$/yr",
    ],
    [
        "Annual air cleaner energy consumption and maintenance costs",
        annual_air_cleaner_total_cost,
        "$/yr",
    ],
    ["Total annual costs", total_annual_cost, "$/yr"],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Annual Carbon Emissions

In [ ]:
# NOTE: The enVerid spreadsheet makes a mistake here!
# The emission factor they use is 7.09E-4 t CO2 / kWh.
# That factor is for electricity reductions!
# We are concerned with emissions associated with consumption, not reduction.
# See https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references

ELECTRICITY_EMISSION_FACTOR = 4.33e-4  # t CO2e / kWh
NG_EMISSION_FACTOR = 0.0053  # t CO2e / therm
STEAM_EMISSION_FACTOR = 0.053  # t CO2e / mmBTU  (assumes perfect efficiency)

# Carbon emissions in t CO2e / yr
cooling_co2_tons = cooling_oa_energy * ELECTRICITY_EMISSION_FACTOR

if oa_heating_src == "Electricity":
    heating_co2_tons = heating_oa_energy * ELECTRICITY_EMISSION_FACTOR
elif oa_heating_src == "Gas":
    heating_co2_tons = heating_oa_energy * NG_EMISSION_FACTOR
elif oa_heating_src == "Steam":
    heating_co2_tons = heating_oa_energy * STEAM_EMISSION_FACTOR

carbon_emiss_oa_vent_energy = cooling_co2_tons + heating_co2_tons
carbon_emiss_filt_fan_energy = annual_filtration_energy * ELECTRICITY_EMISSION_FACTOR
carbon_emiss_air_cleaner_energy = (
    air_cleaner_annual_energy * ELECTRICITY_EMISSION_FACTOR
)

total_carbon_emissions = (
    carbon_emiss_oa_vent_energy
    + carbon_emiss_filt_fan_energy
    + carbon_emiss_air_cleaner_energy
)

disp_table = [
    [
        "Carbon emissions - outside air ventilation energy",
        carbon_emiss_oa_vent_energy,
        "metric tons of CO2/yr",
    ],
    [
        "Carbon emissions - filtration fan energy",
        carbon_emiss_filt_fan_energy,
        "metric tons of CO2/yr",
    ],
    [
        "Carbon emissions - air cleaner energy",
        carbon_emiss_oa_vent_energy,
        "metric tons of CO2/yr",
    ],
    ["Total Carbon Emissions", total_carbon_emissions, "metric tons of CO2/yr"],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")